In [18]:
# Read data from h5 files and calculate moment arms
# Read all the h5 files in the folder
# Write the muscle list that have moment arm for each joint together with their strength and moment arms

import os
import h5py
import pandas as pd

# Specify the directory containing the .h5 files
directory_path = r'C:\Users\hs\Documents\AnyBody models\AMMR4-Beta\Application\Validation\EvaluateMomentArms\Output'

def traverse_h5_group(group, path='', angular_velocity=None):
    """
    Traverse an HDF5 group recursively and collect data for each muscle.

    :param group: h5py.Group
        The HDF5 group to traverse.
    :param path: str
        The path used for recursive traversal.
    :param angular_velocity: float
        Angular velocity to be used for moment arm calculation.
    :return: list
        A list of dictionaries containing muscle data.
    """
    data = []

    for key in group.keys():
        item = group[key]
        new_path = f"{path}.{key}" if path else key
        if isinstance(item, h5py.Group):
            data.extend(traverse_h5_group(item, new_path, angular_velocity))
        elif isinstance(item, h5py.Dataset):
            if key == 'Strength':
                # Process muscle name by removing 'Strength' and adding ';' at the end
                muscle_name = new_path.replace('.Strength', '')
                
                # Get the first value of the Strength array
                strength_array = item[()]
                if strength_array.size > 0:
                    strength = strength_array[0]  # Take only the first value
                else:
                    strength = None  # Handle cases where the array might be empty
                
                # Check if there's an 'LmtDot' dataset in the same group
                lmt_dot_array = None
                if 'LmtDot' in group:
                    lmt_dot_array = group['LmtDot'][()]  # Get the array directly

                # Initialize a dictionary to hold the muscle data
                entry = {'MuscleName': muscle_name, 'Strength': strength}
                
                # Add LmtDot values as separate columns if angular_velocity is available
                if lmt_dot_array is not None and angular_velocity is not None:
                    for i, lmt_dot in enumerate(lmt_dot_array):
                        if angular_velocity != 0:  # Prevent division by zero
                            entry[f'MomentArm_{i}'] = -lmt_dot / angular_velocity
                        else:
                            entry[f'MomentArm_{i}'] = None  # Handle division by zero case
                
                data.append(entry)

    return data

def get_angular_velocity(file_path):
    """
    Retrieve the angular velocity from the HDF5 file.

    :param file_path: str
        Path to the HDF5 file.
    :return: float or None
        Angular velocity if found, otherwise None.
    """
    try:
        with h5py.File(file_path, 'r') as h5_file:
            angular_velocity_path = 'Output/AngularVelocity_output/Val'
            # Ensure we are accessing a dataset, not a group
            if angular_velocity_path in h5_file:
                angular_velocity_dataset = h5_file[angular_velocity_path]
                # Access the dataset value, assuming it might be an array or a single value
                if isinstance(angular_velocity_dataset, h5py.Dataset):
                    # Retrieve the first value if it's an array
                    if angular_velocity_dataset.shape:
                        angular_velocity = angular_velocity_dataset[0]
                    else:
                        angular_velocity = angular_velocity_dataset[()]
                    return angular_velocity
                else:
                    print(f"'{angular_velocity_path}' is not a dataset.")
            else:
                print(f"Angular velocity path '{angular_velocity_path}' not found in {file_path}")

    except Exception as e:
        print(f"Error reading angular velocity from {file_path}: {e}")
    return None

def process_h5_file(file_path):
    """
    Process an HDF5 file and return a DataFrame with muscle data.

    :param file_path: str
        Path to the HDF5 file.
    :return: pd.DataFrame
        DataFrame containing the processed muscle data.
    """
    angular_velocity = get_angular_velocity(file_path)

    with h5py.File(file_path, 'r') as h5_file:
        # Navigate to the specific group
        try:
            base_group = h5_file['Output']['_Main']['HumanModel']['BodyModel']['Trunk']['Muscles']
        except KeyError as e:
            print(f"KeyError: {e}. The file structure is not as expected.")
            return pd.DataFrame()  # Return an empty DataFrame if structure is not as expected

        # Traverse the group and collect data
        muscles_data = traverse_h5_group(base_group, angular_velocity=angular_velocity)

    # Create a DataFrame
    df = pd.DataFrame(muscles_data)

    # Remove columns where all values are None
    df.dropna(axis=1, how='all', inplace=True)

    # Filter out rows where any MomentArm_* column is less than 1e-6 or Strength is None
    moment_arm_cols = [col for col in df.columns if col.startswith('MomentArm_')]
    for col in moment_arm_cols:
        df = df[df[col].apply(lambda x: x is not None and abs(x) >= 1e-6)]

    return df

def process_all_h5_files(directory):
    """
    Process all HDF5 files in a directory and write results to an Excel file.

    :param directory: str
        The directory to search for HDF5 files.
    """
    # Define the mapping for replacements
    replacement_mapping = {
        'Extension': 'FE',
        'AxialRotation': 'AR',
        'LateralBending': 'LB'
    }

    # Create a Pandas Excel writer using openpyxl as the engine
    with pd.ExcelWriter('muscle_MomentArm.xlsx', engine='openpyxl') as writer:
        # Track if any data is written
        data_written = False

        # Iterate over all files in the directory
        for filename in os.listdir(directory):
            if filename.endswith('.h5'):
                # Get the full path to the file
                file_path = os.path.join(directory, filename)
                
                # Process the file
                df = process_h5_file(file_path)
                
                # Only write to Excel if the DataFrame is not empty
                if not df.empty:
                    # Extract the sheet name from the filename and remove '.anydata'
                    sheet_name = os.path.splitext(filename)[0].split('_')[-1].replace('.anydata', '')
                    # Apply the replacement mapping
                    for key, value in replacement_mapping.items():
                        sheet_name = sheet_name.replace(key, value)
                    # Write the DataFrame to a sheet in the Excel file
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    data_written = True
                    print(f"Processed {filename} and written to sheet '{sheet_name}'")
                else:
                    print(f"No valid data found in {filename}, skipping.")

        # Check if any data was written
        if not data_written:
            print("No data was written to the Excel file. Check the input files and directory.")

# Process all HDF5 files in the specified directory
process_all_h5_files(directory_path)


Processed MomentArms_Trunk_C1C0Extension.anydata.h5 and written to sheet 'C1C0FE'
Processed MomentArms_Trunk_C2C1AxialRotation.anydata.h5 and written to sheet 'C2C1AR'
Processed MomentArms_Trunk_C3C2AxialRotation.anydata.h5 and written to sheet 'C3C2AR'
Processed MomentArms_Trunk_C3C2Extension.anydata.h5 and written to sheet 'C3C2FE'
Processed MomentArms_Trunk_C3C2LateralBending.anydata.h5 and written to sheet 'C3C2LB'
Processed MomentArms_Trunk_C4C3AxialRotation.anydata.h5 and written to sheet 'C4C3AR'
Processed MomentArms_Trunk_C4C3Extension.anydata.h5 and written to sheet 'C4C3FE'
Processed MomentArms_Trunk_C4C3LateralBending.anydata.h5 and written to sheet 'C4C3LB'
Processed MomentArms_Trunk_C5C4AxialRotation.anydata.h5 and written to sheet 'C5C4AR'
Processed MomentArms_Trunk_C5C4Extension.anydata.h5 and written to sheet 'C5C4FE'
Processed MomentArms_Trunk_C5C4LateralBending.anydata.h5 and written to sheet 'C5C4LB'
Processed MomentArms_Trunk_C6C5AxialRotation.anydata.h5 and written

In [19]:
# Create AnyScript file with the list of ALL the muscles passes through each joint

import os
import pandas as pd

# Define the path to the Excel file
excel_file = 'muscle_MomentArm.xlsx'

# Define the output file path
output_file_path = 'SpanningMusclesJoints.any'

def process_excel_file(excel_file, output_file_path):
    # Read the Excel file
    xls = pd.ExcelFile(excel_file)
    
    # Dictionary to keep track of processed joints
    processed_joints = {}

    with open(output_file_path, 'w') as file:
        for sheet_name in xls.sheet_names:
            # Extract the joint name (removing the FE, LB, or AR part)
            joint_name = sheet_name.rstrip('FE').rstrip('LB').rstrip('AR')

            # Only process one sheet per joint
            if joint_name not in processed_joints:
                # Read the sheet into a DataFrame
                df = pd.read_excel(xls, sheet_name=sheet_name)
                
                # Ensure there is data in the DataFrame
                if not df.empty:
                    # Extract the muscle names (first column)
                    muscle_names = df.iloc[:, 0].tolist()

                    # Write to the AnyScript file
                    file.write(f'AnyFolder {joint_name}jnt = {{\n')
                    for muscle in muscle_names:
                        file.write(f'    .spine_muscle_folder_side.{muscle};\n')
                    file.write('};\n\n')

                    # Mark the joint as processed
                    processed_joints[joint_name] = sheet_name
                    print(f"Processed joint {joint_name} from sheet {sheet_name}")

process_excel_file(excel_file, output_file_path)


Processed joint C1C0 from sheet C1C0FE
Processed joint C2C1 from sheet C2C1AR
Processed joint C3C2 from sheet C3C2AR
Processed joint C4C3 from sheet C4C3AR
Processed joint C5C4 from sheet C5C4AR
Processed joint C6C5 from sheet C6C5AR
Processed joint C7C6 from sheet C7C6AR
Processed joint L1L2 from sheet L1L2AR
Processed joint L2L3 from sheet L2L3AR
Processed joint L3L4 from sheet L3L4AR
Processed joint L4L5 from sheet L4L5AR
Processed joint L5Sacrum from sheet L5SacrumAR
Processed joint PelvisSacrum from sheet PelvisSacrumAR
Processed joint T10T11 from sheet T10T11AR
Processed joint T11T12 from sheet T11T12AR
Processed joint T12L1 from sheet T12L1AR
Processed joint T1C7 from sheet T1C7AR
Processed joint T1T2 from sheet T1T2AR
Processed joint T2T3 from sheet T2T3AR
Processed joint T3T4 from sheet T3T4AR
Processed joint T4T5 from sheet T4T5AR
Processed joint T5T6 from sheet T5T6AR
Processed joint T6T7 from sheet T6T7AR
Processed joint T7T8 from sheet T7T8AR
Processed joint T8T9 from shee

In [26]:
# Create AnyScript file with the list of EXTENSOR muscles for each joint using a threshold on moment arm values

import os
import pandas as pd

# Define the path to the Excel file
excel_file = 'muscle_MomentArm.xlsx'

# Define the output file path
output_file_path = 'SpanningMusclesJoints_Extensor.any'

def process_excel_file(excel_file, output_file_path):
    # Read the Excel file
    xls = pd.ExcelFile(excel_file)
    
    # Dictionary to keep track of processed joints
    processed_joints = {}

    with open(output_file_path, 'w') as file:
        for sheet_name in xls.sheet_names:
            # Only process sheets with 'FE' suffix
            if sheet_name.endswith('FE'):
                # Extract the joint name by removing the 'FE' suffix
                joint_name = sheet_name[:-2]

                # Read the sheet into a DataFrame
                df = pd.read_excel(xls, sheet_name=sheet_name)
                
                # Ensure there is data in the DataFrame
                if not df.empty:
                    # Filter muscles where all moment arms are greater than 1e-3
                    moment_arm_cols = [col for col in df.columns if col.startswith('MomentArm_')]
                    df_filtered = df[df[moment_arm_cols].gt(1e-7).all(axis=1)]

                    # Extract the muscle names (first column) from the filtered DataFrame
                    muscle_names = df_filtered.iloc[:, 0].tolist()

                    # Write to the AnyScript file
                    file.write(f'AnyFolder {joint_name}jnt = {{\n')
                    for muscle in muscle_names:
                        file.write(f'    spine_muscle_folder_side.{muscle};\n')
                    file.write('};\n\n')

                    # Mark the joint as processed
                    processed_joints[joint_name] = sheet_name
                    print(f"Processed joint {joint_name} from sheet {sheet_name}")

process_excel_file(excel_file, output_file_path)


Processed joint C1C0 from sheet C1C0FE
Processed joint C3C2 from sheet C3C2FE
Processed joint C4C3 from sheet C4C3FE
Processed joint C5C4 from sheet C5C4FE
Processed joint C6C5 from sheet C6C5FE
Processed joint C7C6 from sheet C7C6FE
Processed joint L1L2 from sheet L1L2FE
Processed joint L2L3 from sheet L2L3FE
Processed joint L3L4 from sheet L3L4FE
Processed joint L4L5 from sheet L4L5FE
Processed joint L5Sacrum from sheet L5SacrumFE
Processed joint PelvisSacrum from sheet PelvisSacrumFE
Processed joint T10T11 from sheet T10T11FE
Processed joint T11T12 from sheet T11T12FE
Processed joint T12L1 from sheet T12L1FE
Processed joint T1C7 from sheet T1C7FE
Processed joint T1T2 from sheet T1T2FE
Processed joint T2T3 from sheet T2T3FE
Processed joint T3T4 from sheet T3T4FE
Processed joint T4T5 from sheet T4T5FE
Processed joint T5T6 from sheet T5T6FE
Processed joint T6T7 from sheet T6T7FE
Processed joint T7T8 from sheet T7T8FE
Processed joint T8T9 from sheet T8T9FE
Processed joint T9T10 from she

In [25]:

 
import os
import pandas as pd

# Define the path to the Excel file
excel_file = 'muscle_MomentArm.xlsx'

# Define the output file path
output_file_path = 'SpanningMusclesJoints_Flexor.any'

def process_excel_file(excel_file, output_file_path):
    # Read the Excel file
    xls = pd.ExcelFile(excel_file)
    
    # Dictionary to keep track of processed joints
    processed_joints = {}

    with open(output_file_path, 'w') as file:
        for sheet_name in xls.sheet_names:
            # Only process sheets with 'FE' suffix
            if sheet_name.endswith('FE'):
                # Extract the joint name by removing the 'FE' suffix
                joint_name = sheet_name[:-2]

                # Read the sheet into a DataFrame
                df = pd.read_excel(xls, sheet_name=sheet_name)
                
                # Ensure there is data in the DataFrame
                if not df.empty:
                    # Filter muscles where all moment arms are less than -1e-3
                    moment_arm_cols = [col for col in df.columns if col.startswith('MomentArm_')]
                    df_filtered = df[df[moment_arm_cols].lt(-1e-7).all(axis=1)]

                    # Extract the muscle names (first column) from the filtered DataFrame
                    muscle_names = df_filtered.iloc[:, 0].tolist()

                    # Write to the AnyScript file
                    file.write(f'AnyFolder {joint_name}jnt = {{\n')
                    for muscle in muscle_names:
                        file.write(f'    spine_muscle_folder_side.{muscle};\n')
                    file.write('};\n\n')

                    # Mark the joint as processed
                    processed_joints[joint_name] = sheet_name
                    print(f"Processed joint {joint_name} from sheet {sheet_name}")

process_excel_file(excel_file, output_file_path)


Processed joint C1C0 from sheet C1C0FE
Processed joint C3C2 from sheet C3C2FE
Processed joint C4C3 from sheet C4C3FE
Processed joint C5C4 from sheet C5C4FE
Processed joint C6C5 from sheet C6C5FE
Processed joint C7C6 from sheet C7C6FE
Processed joint L1L2 from sheet L1L2FE
Processed joint L2L3 from sheet L2L3FE
Processed joint L3L4 from sheet L3L4FE
Processed joint L4L5 from sheet L4L5FE
Processed joint L5Sacrum from sheet L5SacrumFE
Processed joint PelvisSacrum from sheet PelvisSacrumFE
Processed joint T10T11 from sheet T10T11FE
Processed joint T11T12 from sheet T11T12FE
Processed joint T12L1 from sheet T12L1FE
Processed joint T1C7 from sheet T1C7FE
Processed joint T1T2 from sheet T1T2FE
Processed joint T2T3 from sheet T2T3FE
Processed joint T3T4 from sheet T3T4FE
Processed joint T4T5 from sheet T4T5FE
Processed joint T5T6 from sheet T5T6FE
Processed joint T6T7 from sheet T6T7FE
Processed joint T7T8 from sheet T7T8FE
Processed joint T8T9 from sheet T8T9FE
Processed joint T9T10 from she

In [24]:
import os
import pandas as pd

# Define the path to the Excel file
excel_file = 'muscle_MomentArm.xlsx'

def check_muscle_groups_by_joint(excel_file):
    # Read the Excel file
    xls = pd.ExcelFile(excel_file)
    
    # Dictionary to keep track of processed joints
    muscle_groups_by_joint = {}

    for sheet_name in xls.sheet_names:
        # Only process sheets with 'FE' suffix
        if sheet_name.endswith('FE'):
            # Extract the joint name by removing the 'FE' suffix
            joint_name = sheet_name[:-2]

            # Read the sheet into a DataFrame
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Ensure there is data in the DataFrame
            if not df.empty:
                # Filter muscles where all moment arms are less than -1e-3
                moment_arm_cols = [col for col in df.columns if col.startswith('MomentArm_')]
                df_less_than_neg = df[df[moment_arm_cols].lt(-1e-3).all(axis=1)]
                muscles_less_than_neg = set(df_less_than_neg.iloc[:, 0].tolist())
                
                # Filter muscles where all moment arms are greater than 1e-3
                df_greater_than_pos = df[df[moment_arm_cols].gt(1e-3).all(axis=1)]
                muscles_greater_than_pos = set(df_greater_than_pos.iloc[:, 0].tolist())

                # Store the muscle sets in the dictionary
                muscle_groups_by_joint[joint_name] = {
                    'less_than_neg': muscles_less_than_neg,
                    'greater_than_pos': muscles_greater_than_pos
                }

    # Check for overlaps in each joint
    for joint, muscle_groups in muscle_groups_by_joint.items():
        overlap_muscles = muscle_groups['less_than_neg'].intersection(muscle_groups['greater_than_pos'])
        
        if overlap_muscles:
            print(f"Overlap found in joint {joint}! The following muscles are in both groups:")
            for muscle in overlap_muscles:
                print(f"- {muscle}")
        else:
            print(f"No overlap found in joint {joint}. The groups are mutually exclusive.")

# Run the check
check_muscle_groups_by_joint(excel_file)


No overlap found in joint C1C0. The groups are mutually exclusive.
No overlap found in joint C3C2. The groups are mutually exclusive.
No overlap found in joint C4C3. The groups are mutually exclusive.
No overlap found in joint C5C4. The groups are mutually exclusive.
No overlap found in joint C6C5. The groups are mutually exclusive.
No overlap found in joint C7C6. The groups are mutually exclusive.
No overlap found in joint L1L2. The groups are mutually exclusive.
No overlap found in joint L2L3. The groups are mutually exclusive.
No overlap found in joint L3L4. The groups are mutually exclusive.
No overlap found in joint L4L5. The groups are mutually exclusive.
No overlap found in joint L5Sacrum. The groups are mutually exclusive.
No overlap found in joint PelvisSacrum. The groups are mutually exclusive.
No overlap found in joint T10T11. The groups are mutually exclusive.
No overlap found in joint T11T12. The groups are mutually exclusive.
No overlap found in joint T12L1. The groups ar

In [27]:
import os
import pandas as pd

# Define the path to the Excel file
excel_file = 'muscle_MomentArm.xlsx'

def check_muscle_groups_by_joint(excel_file):
    # Read the Excel file
    xls = pd.ExcelFile(excel_file)
    
    # Dictionary to keep track of processed joints
    muscle_groups_by_joint = {}

    for sheet_name in xls.sheet_names:
        # Only process sheets with 'FE' suffix
        if sheet_name.endswith('FE'):
            # Extract the joint name by removing the 'FE' suffix
            joint_name = sheet_name[:-2]

            # Read the sheet into a DataFrame
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Ensure there is data in the DataFrame
            if not df.empty:
                # Filter muscles where all moment arms are less than -1e-3
                moment_arm_cols = [col for col in df.columns if col.startswith('MomentArm_')]
                df_less_than_neg = df[df[moment_arm_cols].lt(-1e-3).all(axis=1)]
                muscles_less_than_neg = set(df_less_than_neg.iloc[:, 0].tolist())
                
                # Filter muscles where all moment arms are greater than 1e-3
                df_greater_than_pos = df[df[moment_arm_cols].gt(1e-3).all(axis=1)]
                muscles_greater_than_pos = set(df_greater_than_pos.iloc[:, 0].tolist())

                # Store the muscle sets in the dictionary
                muscle_groups_by_joint[joint_name] = {
                    'less_than_neg': muscles_less_than_neg,
                    'greater_than_pos': muscles_greater_than_pos
                }

    # Check for overlaps in each joint
    for joint, muscle_groups in muscle_groups_by_joint.items():
        overlap_muscles = muscle_groups['less_than_neg'].intersection(muscle_groups['greater_than_pos'])
        
        if overlap_muscles:
            print(f"Overlap found in joint {joint}! The following muscles are in both groups:")
            for muscle in overlap_muscles:
                print(f"- {muscle}")
        else:
            print(f"No overlap found in joint {joint}. The groups are mutually exclusive.")

# Run the check
check_muscle_groups_by_joint(excel_file)


No overlap found in joint C1C0. The groups are mutually exclusive.
No overlap found in joint C3C2. The groups are mutually exclusive.
No overlap found in joint C4C3. The groups are mutually exclusive.
No overlap found in joint C5C4. The groups are mutually exclusive.
No overlap found in joint C6C5. The groups are mutually exclusive.
No overlap found in joint C7C6. The groups are mutually exclusive.
No overlap found in joint L1L2. The groups are mutually exclusive.
No overlap found in joint L2L3. The groups are mutually exclusive.
No overlap found in joint L3L4. The groups are mutually exclusive.
No overlap found in joint L4L5. The groups are mutually exclusive.
No overlap found in joint L5Sacrum. The groups are mutually exclusive.
No overlap found in joint PelvisSacrum. The groups are mutually exclusive.
No overlap found in joint T10T11. The groups are mutually exclusive.
No overlap found in joint T11T12. The groups are mutually exclusive.
No overlap found in joint T12L1. The groups ar

In [28]:
# Find flexor and extensor muscles for each joint using a threshold on moment arm values and check there is no overlap betweent them.
# Create AnyScript file with the list of EXTENSOR and FLEXOR muscles for each joint.

import os
import pandas as pd

# Define the path to the Excel file
excel_file = 'muscle_MomentArm.xlsx'

# Define the output file paths
extensor_file_path = 'SpanningMusclesJoints_Extensor.any'
flexor_file_path = 'SpanningMusclesJoints_Flexor.any'

def process_excel_file_for_muscle_types(excel_file, extensor_file_path, flexor_file_path):
    # Read the Excel file
    xls = pd.ExcelFile(excel_file)
    
    # Dictionary to keep track of processed joints
    muscle_groups_by_joint = {}

    for sheet_name in xls.sheet_names:
        # Only process sheets with 'FE' suffix
        if sheet_name.endswith('FE'):
            # Extract the joint name by removing the 'FE' suffix
            joint_name = sheet_name[:-2]

            # Read the sheet into a DataFrame
            df = pd.read_excel(xls, sheet_name=sheet_name)
            
            # Ensure there is data in the DataFrame
            if not df.empty:
                # Filter muscles where all moment arms are less than -1e-3
                moment_arm_cols = [col for col in df.columns if col.startswith('MomentArm_')]
                df_less_than_neg = df[df[moment_arm_cols].lt(-1e-3).all(axis=1)]
                muscles_less_than_neg = set(df_less_than_neg.iloc[:, 0].tolist())
                
                # Filter muscles where all moment arms are greater than 1e-3
                df_greater_than_pos = df[df[moment_arm_cols].gt(1e-3).all(axis=1)]
                muscles_greater_than_pos = set(df_greater_than_pos.iloc[:, 0].tolist())

                # Store the muscle sets in the dictionary
                muscle_groups_by_joint[joint_name] = {
                    'less_than_neg': muscles_less_than_neg,
                    'greater_than_pos': muscles_greater_than_pos
                }

    # Write Extensor muscles to the AnyScript file
    with open(extensor_file_path, 'w') as file:
        for joint, muscle_groups in muscle_groups_by_joint.items():
            file.write(f'AnyFolder {joint}jnt = {{\n')
            for muscle in muscle_groups['greater_than_pos']:
                file.write(f'    spine_muscle_folder_side.{muscle};\n')
            file.write('};\n\n')
            print(f"Processed joint {joint} for Extensor muscles")

    # Write Flexor muscles to the AnyScript file
    with open(flexor_file_path, 'w') as file:
        for joint, muscle_groups in muscle_groups_by_joint.items():
            file.write(f'AnyFolder {joint}jnt = {{\n')
            for muscle in muscle_groups['less_than_neg']:
                file.write(f'    spine_muscle_folder_side.{muscle};\n')
            file.write('};\n\n')
            print(f"Processed joint {joint} for Flexor muscles")

# Run the processing
process_excel_file_for_muscle_types(excel_file, extensor_file_path, flexor_file_path)


Processed joint C1C0 for Extensor muscles
Processed joint C3C2 for Extensor muscles
Processed joint C4C3 for Extensor muscles
Processed joint C5C4 for Extensor muscles
Processed joint C6C5 for Extensor muscles
Processed joint C7C6 for Extensor muscles
Processed joint L1L2 for Extensor muscles
Processed joint L2L3 for Extensor muscles
Processed joint L3L4 for Extensor muscles
Processed joint L4L5 for Extensor muscles
Processed joint L5Sacrum for Extensor muscles
Processed joint PelvisSacrum for Extensor muscles
Processed joint T10T11 for Extensor muscles
Processed joint T11T12 for Extensor muscles
Processed joint T12L1 for Extensor muscles
Processed joint T1C7 for Extensor muscles
Processed joint T1T2 for Extensor muscles
Processed joint T2T3 for Extensor muscles
Processed joint T3T4 for Extensor muscles
Processed joint T4T5 for Extensor muscles
Processed joint T5T6 for Extensor muscles
Processed joint T6T7 for Extensor muscles
Processed joint T7T8 for Extensor muscles
Processed joint T